In [76]:
# Load the senegal dataset
import pandas as pd
import numpy as np
import os
os.chdir('..')
#l = pd.read_csv('/data/senegal_weather_weekly_2010_2019.csv')
#weather_8day = pd.read_csv('/data/senegal_weather_8days_2010_2019.csv')
weather_mon = pd.read_csv('/data/monthly_senegal_data.csv')

# l.head()
weather_mon.tail()


,ind,year,month,gpslat,gpslon,temp,rel_hum,hum,wind_speed,wind_dir,solar_radiat,thermal_radiat,dir_solar_radiat,soil_1,soil_2,soil_3,soil_4,pr
97795,9775,2019,8,13.826203,-16.13396,28.388571,0.761786,39.382500,2.773571,246.125000,216.178571,422.178571,133.938929,30.083571,29.697500,29.865357,29.461071,0.322500
97796,9776,2019,9,13.826203,-16.13396,27.662000,0.850857,39.920286,1.645714,223.943429,191.657143,425.000000,104.447714,28.384286,28.046857,28.349143,29.146000,0.304000
97797,9777,2019,10,13.826203,-16.13396,28.340714,0.797500,40.156071,1.820357,234.056429,224.357143,414.071429,155.474286,29.078929,28.707857,28.718571,28.866429,0.089286
97798,9778,2019,11,13.826203,-16.13396,27.861786,0.514286,33.137857,1.679643,194.982857,242.035714,381.821429,189.776786,28.389643,28.643571,28.705000,28.777857,0.001071
97799,9779,2019,12,13.826203,-16.13396,25.666571,0.394571,27.382000,2.604286,86.030000,224.742857,348.142857,167.827714,25.691143,26.242857,27.202000,28.450000,0.000000


Alternative methods

In [77]:
# Dropping duplicate weather data
weather_mon_wodup = weather_mon.drop_duplicates(['gpslat','gpslon','year','month'],keep='first')
print(weather_mon_wodup.shape)

(91680, 18)


In [78]:
# Adding an 'index' for time so that the values are continuously increasing
weather_mon_wodup['month_ind']=12*(weather_mon_wodup['year']-2010)+weather_mon_wodup['month']
weather_mon_wodup.tail()

C:\Anaconda3\envs\omdena-GPSDD\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ind,year,month,gpslat,gpslon,temp,rel_hum,hum,wind_speed,wind_dir,solar_radiat,thermal_radiat,dir_solar_radiat,soil_1,soil_2,soil_3,soil_4,pr,month_ind
97795,9775,2019,8,13.826203,-16.13396,28.388571,0.761786,39.382500,2.773571,246.125000,216.178571,422.178571,133.938929,30.083571,29.697500,29.865357,29.461071,0.322500,116
97796,9776,2019,9,13.826203,-16.13396,27.662000,0.850857,39.920286,1.645714,223.943429,191.657143,425.000000,104.447714,28.384286,28.046857,28.349143,29.146000,0.304000,117
97797,9777,2019,10,13.826203,-16.13396,28.340714,0.797500,40.156071,1.820357,234.056429,224.357143,414.071429,155.474286,29.078929,28.707857,28.718571,28.866429,0.089286,118
97798,9778,2019,11,13.826203,-16.13396,27.861786,0.514286,33.137857,1.679643,194.982857,242.035714,381.821429,189.776786,28.389643,28.643571,28.705000,28.777857,0.001071,119
97799,9779,2019,12,13.826203,-16.13396,25.666571,0.394571,27.382000,2.604286,86.030000,224.742857,348.142857,167.827714,25.691143,26.242857,27.202000,28.450000,0.000000,120


In [74]:
# Getting final unique location (lat, lon) info
ipar_wodup = weather_mon.drop_duplicates(['gpslat','gpslon'],keep='first')
ipar_wodup.shape

(764, 18)

### Convert csv format to netCDF

In [18]:
!pip install netCDF4

In [101]:
# Creating netCDF file for precipitation

import pandas as pd
import numpy as np
import netCDF4


lons = ipar_wodup['gpslon'] 
lats = ipar_wodup['gpslat'] 
nstations = ipar_wodup.shape[0]

lons_out = lons.tolist()
lats_out = lats.tolist()
#time_out = #orig.index.tolist()

# time_out = pd.to_datetime(weather_8day_ipar_wodup['year'].astype(str), format='%Y') + \
#              pd.to_timedelta(weather_8day_ipar_wodup['week'].mul(8).astype(str) + ' days')

#time_out = weather_8day_ipar_wodup.drop_duplicates('day_of_obsv',keep='first')['day_of_obsv']
time_out = weather_mon_wodup.drop_duplicates('month_ind',keep='first')['month_ind']
ntimeintv = time_out.size

print(nstations)
print(ntimeintv)

ncout = netCDF4.Dataset('Precip_2010_2019_monthly.nc', 'w')

ncout.createDimension('division',nstations)
ncout.createDimension('time',ntimeintv)

lats = ncout.createVariable('latitude',np.dtype('float32').char,('division',))
lons = ncout.createVariable('longitude',np.dtype('float32').char,('division',))
time = ncout.createVariable('time',np.dtype('float32').char,('time',))

# required format of weather data for using climate_indices Python package
# "Valid dimension names and order: 
# [('lat', 'lon', 'time'), ('time', 'lat', 'lon'), ('time', 'division'), ('division', 'time')]"
# the format for temp & precip data as 'time' & 'division'. 
# The climate_indices package takes input as  'time' & 'division' or 'time', 'lat' & 'lon'. 
# The latter is possible only for gridded locations. In our case the IPAR data is not gridded so I used the former format. 

precip = ncout.createVariable('precip',np.dtype('float32').char,('time', 'division'))

lats[:] = lats_out
lons[:] = lons_out
time[:] = time_out

print(precip.shape)
# reshape the precipitaiton data to the right dimension 
# - time x station in this case
precip[:] = np.reshape(weather_mon_wodup['pr'].to_numpy(),(ntimeintv,nstations)) 
ncout.close()

764
120
(120, 764)


In [99]:
# Creating netCDF file for temperature

ncout = netCDF4.Dataset('Temp_2010_2019_monthly.nc', 'w')

ncout.createDimension('division',nstations)
ncout.createDimension('time',ntimeintv)

lats = ncout.createVariable('latitude',np.dtype('float32').char,('division',))
lons = ncout.createVariable('longitude',np.dtype('float32').char,('division',))
time = ncout.createVariable('time',np.dtype('float32').char,('time',))
temp = ncout.createVariable('temp',np.dtype('float32').char,('time', 'division'))

lats[:] = lats_out
lons[:] = lons_out
time[:] = time_out
temp[:] = np.reshape(weather_mon_wodup['temp'].to_numpy(),(ntimeintv,nstations)) 
ncout.close()

In [98]:
ncout.close()

### Get Climate Indices 

In [89]:
conda install -c conda-forge nco

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda3\envs\omdena-GPSDD

  added / updated specs:
    - nco


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py37h03978a9_1         143 KB  conda-forge
    gsl-2.6                    |       hdfb1a43_1         1.6 MB  conda-forge
    nco-4.9.7                  |       h38770fd_0         2.5 MB  conda-forge
    openssl-1.1.1i             |       h8ffe710_0         5.8 MB  conda-forge
    udunits2-2.2.27.27         |       h0cdffdb_0         126 KB  conda-forge
Note: you may need to restart the kernel to use updated packages.
    ------------------------------------------------------------
                                           Total:        10.1 MB

The following NEW packages will be INSTALLED:

  gsl                conda-forge/win-64::gsl-2.6-hdfb1a43_1
  nco   

In [85]:
!pip install climate-indices

In [92]:
!wsl which bash

'wsl' is not recognized as an internal or external command,
operable program or batch file.


In [103]:
%%cmd
process_climate_indices --index spei --periodicity monthly --netcdf_precip Precip_2010_2019_monthly.nc --var_name_precip  precip --netcdf_temp Temp_2010_2019_monthly.nc --var_name_temp temp --output_file_base spei_2010_2019_monthly --scales 9 18 --calibration_start_year 2010 --calibration_end_year 2019

Microsoft Windows [Version 10.0.18363.1256]
(c) 2019 Microsoft Corporation. All rights reserved.

(omdena-GPSDD) C:\Sandeep\Learning\Omdena\GPSDD\notebooks>process_climate_indices --index spei --periodicity monthly --netcdf_precip Precip_2010_2019_monthly.nc --var_name_precip  precip --netcdf_temp Temp_2010_2019_monthly.nc --var_name_temp temp --output_file_base spei_2010_2019_monthly --scales 9 18 --calibration_start_year 2010 --calibration_end_year 2019
Error in calling operator ncpdq with:
>>> C:\Anaconda3\envs\omdena-GPSDD\Library\bin\ncpdq -a division,time -O --output=C:\Users\mudig\AppData\Local\Temp\ncpdq_Temp_2010_2019_monthly.ncxlr5c650.tmp Temp_2010_2019_monthly.nc <<<
Inputs: Temp_2010_2019_monthly.nc
b'Access is denied.\r\n'

(omdena-GPSDD) C:\Sandeep\Learning\Omdena\GPSDD\notebooks>

2021-01-08  01:00:34 INFO Start time:    2021-01-08 01:00:34.670241
2021-01-08  01:00:34 ERROR Failed to complete
Traceback (most recent call last):
  File "c:\anaconda3\envs\omdena-gpsdd\lib\site-packages\climate_indices\__main__.py", line 1698, in main
    arguments.netcdf_temp, arguments.var_name_temp
  File "c:\anaconda3\envs\omdena-gpsdd\lib\site-packages\climate_indices\__main__.py", line 1534, in _prepare_file
    options=[f'-a "{dims}"', "-O"])
  File "c:\anaconda3\envs\omdena-gpsdd\lib\site-packages\nco\nco.py", line 342, in get
    raise NCOException(**retvals)
nco.nco.NCOException: (returncode:1) b'Access is denied.\r\n'
Traceback (most recent call last):
  File "c:\anaconda3\envs\omdena-gpsdd\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\anaconda3\envs\omdena-gpsdd\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\omdena-GPSDD\Scripts\process_climate_indices.exe\__main__.py", line 7, in <module>

### Some trials w 8-day data (not required for monthly data)

In [8]:
# get weather location coordinates

ipar_yield = pd.read_excel('./data/yields_ndvi_no_duplicates.xls',sheet_name='Maize')
ipar_locations = ipar_yield[['gpslat', 'gpslon']]
ipar_locations = ipar_locations[ipar_locations['gpslat']>0]

print(ipar_locations.shape)
ipar_locations.head()

(767, 2)


,gpslat,gpslon
0,13.988077,-15.839605
1,12.749203,-16.049720
3,12.788813,-15.584448
4,13.257797,-13.801437
5,13.065142,-15.624235


In [3]:
weather_8day['day_of_obsv'] = pd.to_datetime(weather_8day['year'].astype(str), format='%Y') + \
             pd.to_timedelta(weather_8day['nth_8days'].mul(8).astype(str) + ' days')

weather_8day[weather_8day['year']==2018].head()

,year,nth_8days,gpslat,gpslon,temp,rel_hum,hum,wind_speed,wind_dir,solar_radiat,thermal_radiat,dir_solar_radiat,soil_1,soil_2,soil_3,soil_4,pr,day_of_obsv
368,2018,1,12.749203,-16.04972,24.49250,0.40875,25.90000,2.75000,27.14250,229.000,339.000,168.62875,24.38875,24.66125,25.69000,27.73500,0.00000,2018-01-09
369,2018,2,12.749203,-16.04972,24.34375,0.37125,25.07750,2.91000,26.52000,243.125,328.375,189.20625,24.13750,24.56625,25.39750,27.42250,0.00000,2018-01-17
370,2018,3,12.749203,-16.04972,25.02500,0.33750,25.40875,3.09625,36.49375,244.000,338.000,188.71625,24.61875,24.74750,25.25000,27.12625,0.00000,2018-01-25
371,2018,4,12.749203,-16.04972,24.12750,0.34875,24.38500,3.05500,98.05500,235.250,342.250,159.23750,24.56000,24.98125,25.54875,26.89000,0.00000,2018-02-02
372,2018,5,12.749203,-16.04972,25.06250,0.35500,25.80250,3.81375,21.96625,218.500,366.250,142.93500,25.10500,25.07250,25.37625,26.68250,0.00375,2018-02-10


In [4]:
print(weather_8day[weather_8day['year']==2015]['day_of_obsv'].size)
print(weather_8day[weather_8day['year']==2016]['day_of_obsv'].size)

68310
68310


In [20]:
l[l['year']==2016]['gpslat'].describe()

count    42380.000000
mean        13.479826
std          0.811272
min         12.388267
25%         12.896433
50%         13.214038
75%         13.836808
max         16.514580
Name: gpslat, dtype: float64

In [9]:
weather_8day_ipar = weather_8day.merge(ipar_locations,on=['gpslat','gpslon'],how='inner')
weather_8day_ipar.shape

(505540, 18)

In [10]:
weather_8day_ipar.head()

,year,nth_8days,gpslat,gpslon,temp,rel_hum,hum,wind_speed,wind_dir,solar_radiat,thermal_radiat,dir_solar_radiat,soil_1,soil_2,soil_3,soil_4,pr,day_of_obsv
0,2010,1,12.749203,-16.04972,24.70250,0.46500,27.25375,2.20250,49.04125,225.125,343.750,161.05125,24.93625,25.29625,25.78125,27.05375,0.0000,2010-01-09
1,2010,2,12.749203,-16.04972,24.67750,0.40000,26.07625,2.73000,38.74875,227.500,340.000,159.82750,24.30250,24.58500,25.39125,26.85375,0.0000,2010-01-17
2,2010,3,12.749203,-16.04972,25.39250,0.37625,26.63250,2.01250,85.32500,241.000,338.500,176.55375,25.06250,25.15875,25.33375,26.63625,0.0000,2010-01-25
3,2010,4,12.749203,-16.04972,25.62375,0.46125,28.60250,1.86500,159.42875,235.250,358.875,165.79500,25.89750,25.79500,25.63125,26.47250,0.0025,2010-02-02
4,2010,5,12.749203,-16.04972,27.66000,0.42875,30.95250,2.13125,89.56500,241.250,359.250,163.56500,27.32250,26.97125,26.43250,26.41000,0.0000,2010-02-10


In [17]:
weather_8day_ipar_wodup = weather_8day_ipar.drop_duplicates(['gpslat','gpslon','year','nth_8days'],keep='first')
weather_8day_ipar_wodup.shape

(322920, 18)

##### Trial to aggregate 8-day data to month

In [54]:
weather_month_ipar_wodup = weather_8day_ipar_wodup.groupby([weather_8day_ipar_wodup['day_of_obsv'].dt.month,'gpslat','gpslon','year']).agg({'pr':['sum']})
weather_month_ipar_wodup.columns

MultiIndex([('pr', 'sum')],
           )

In [55]:
weather_month_ipar_wodup.columns = ['pr_month']
weather_month_ipar_wodup = weather_month_ipar_wodup.reset_index()
weather_month_ipar_wodup.head()

,day_of_obsv,gpslat,gpslon,year,pr_month
0,1,12.388267,-16.738473,2010,0.00000
1,1,12.388267,-16.738473,2011,0.00000
2,1,12.388267,-16.738473,2012,0.00000
3,1,12.388267,-16.738473,2013,0.00125
4,1,12.388267,-16.738473,2014,0.00000


In [59]:
weather_month_ipar_wodup.shape
weather_month_ipar_wodup.rename(columns={'day_of_obsv':'month'},inplace=True)
weather_month_ipar_wodup.head()

,month,gpslat,gpslon,year,pr_month
0,1,12.388267,-16.738473,2010,0.00000
1,1,12.388267,-16.738473,2011,0.00000
2,1,12.388267,-16.738473,2012,0.00000
3,1,12.388267,-16.738473,2013,0.00125
4,1,12.388267,-16.738473,2014,0.00000


In [69]:
weather_month_ipar_wodup['month_ind']=12*(weather_month_ipar_wodup['year']-2010)+weather_month_ipar_wodup['month']
# weather_month_ipar_wodup.sort
weather_month_ipar_wodup.head()

,month,gpslat,gpslon,year,pr_month,month_ind
0,1,12.388267,-16.738473,2010,0.00000,1
1,1,12.388267,-16.738473,2011,0.00000,13
2,1,12.388267,-16.738473,2012,0.00000,25
3,1,12.388267,-16.738473,2013,0.00125,37
4,1,12.388267,-16.738473,2014,0.00000,49
